In [2]:
from spektral.datasets import TUDataset
from spektral.transforms import GCNFilter
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool
from spektral.data import BatchLoader



## Importar datos

In [3]:
data = TUDataset('PROTEINS')

data

Successfully loaded PROTEINS.


TUDataset(n_graphs=1113)

## preprocesamiento

In [8]:
data.apply(GCNFilter())
X_train, X_test=train_test_split(data, test_size=0.20, random_state=42)


## Creacion de layers

In [ ]:
class ProteinsGNN(Model):
  
  def __init__(self, n_hidden, n_labels):
    super().__init__()
    # Define our GCN layer with our n_hidden layers
    self.graph_conv = GCNConv(n_hidden)
    # Define our global pooling layer
    self.pool = GlobalSumPool()
    # Define our dropout layer, initialize dropout freq. to .5 (50%)
    self.dropout = Dropout(0.5)
    # Define our Dense layer, with softmax activation function
    self.dense = Dense(n_labels, 'softmax')

# Define class method to call model on input
  def call(self, inputs):
    out = self.graph_conv(inputs)
    out = self.dropout(out)
    out = self.pool(out)
    out = self.dense(out)

    return out

## creacion del modelo 

In [ ]:
model = ProteinsGNN(32, data.n_labels)

model.compile('adam', 'categorical_crossentropy')

loader = BatchLoader(data_train, batch_size=32)

## Entrenar el modelo 


In [ ]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

## Evaluar el modelo 

In [ ]:
test_loader = BatchLoader(data_test, batch_size=32)

loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))